In [1]:
import torch
import random
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
from transformers import logging

logging.set_verbosity_error()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
def cos(a, b):
    return a.dot(b) / (a.dot(a) * b.dot(b)) ** 0.5

### a)

In [3]:
model_name = 'flax-community/papuGaPT2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def tokenize(word):
    ids = tokenizer(word, return_tensors='pt')['input_ids'][0]
    return [tokenizer.decode(n) for n in ids]

emb = model.transformer.wte.weight.detach().cpu().numpy()

tokenizer_config.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/888k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [4]:
def get_context_free_token_embedding(word):
    token_ids = tokenizer.encode(' ' + word)
    emb[token_ids[0]] *= 3
    whole_word_embbeding = np.sum(np.stack([emb[id] for id in token_ids]), axis=0) / len(token_ids)
    return whole_word_embbeding

### b)

In [5]:
model_name_encoder = "allegro/herbert-base-cased"
tokenizer_encoder = AutoTokenizer.from_pretrained(model_name_encoder)
model_encoder = AutoModel.from_pretrained(model_name_encoder).to(device)

In [6]:
def get_context_token_embedding(word):
    input_ids = tokenizer_encoder(word, return_tensors='pt')['input_ids'].to(device)
    output = model_encoder(input_ids=input_ids)
    return output.last_hidden_state.detach().cpu().numpy()[0,0,:]

### spoil funcion

In [7]:
def spoil(word):
    replacements = {'ł': 'l', 'ą': 'a', 'ć': 'c', 'ę': 'e', 'ń': 'n', 'ó': 'o', 'ś': 's', 'ź': 'z', 'ż': 'z'}
    new_word = ''.join(replacements.get(c, c) for c in word)
    if new_word != word:
        idx = random.randrange(0, len(new_word) - 1)
        temp = list(new_word)
        temp[idx], temp[idx + 1] = temp[idx + 1], temp[idx]
        new_word = ''.join(temp)
    return new_word


### testing

In [18]:

clusters_txt = '''piśmiennicze: pisak flamaster ołówek długopis pióro
małe_ssaki: mysz szczur chomik łasica kuna bóbr
okręty: niszczyciel lotniskowiec trałowiec krążownik pancernik fregata korweta
lekarze: lekarz pediatra ginekolog kardiolog internista geriatra
zupy: rosół żurek barszcz
uczucia: miłość przyjaźń nienawiść gniew smutek radość strach
działy_matematyki: algebra analiza topologia logika geometria 
budynki_sakralne: kościół bazylika kaplica katedra świątynia synagoga zbór
stopień_wojskowy: chorąży podporucznik porucznik kapitan major pułkownik generał podpułkownik
grzyby_jadalne: pieczarka borowik gąska kurka boczniak kania
prądy_filozoficzne: empiryzm stoicyzm racjonalizm egzystencjalizm marksizm romantyzm
religie: chrześcijaństwo buddyzm islam prawosławie protestantyzm kalwinizm luteranizm judaizm
dzieła_muzyczne: sonata synfonia koncert preludium fuga suita
cyfry: jedynka dwójka trójka czwórka piątka szóstka siódemka ósemka dziewiątka
owady: ważka biedronka żuk mrówka mucha osa pszczoła chrząszcz
broń_biała: miecz topór sztylet nóż siekiera
broń_palna: karabin pistolet rewolwer fuzja strzelba
komputery: komputer laptop kalkulator notebook
kolory: biel żółć czerwień błękit zieleń brąz czerń
duchowny: wikary biskup ksiądz proboszcz rabin pop arcybiskup kardynał pastor
ryby: karp śledź łosoś dorsz okoń sandacz szczupak płotka
napoje_mleczne: jogurt kefir maślanka
czynności_sportowe: bieganie skakanie pływanie maszerowanie marsz trucht
ubranie:  garnitur smoking frak żakiet marynarka koszula bluzka sweter sweterek sukienka kamizelka spódnica spodnie
mebel: krzesło fotel kanapa łóżko wersalka sofa stół stolik ława
przestępca: morderca zabójca gwałciciel złodziej bandyta kieszonkowiec łajdak łobuz
mięso_wędliny: wieprzowina wołowina baranina cielęcina boczek baleron kiełbasa szynka schab karkówka dziczyzna
drzewo: dąb klon wiąz jesion świerk sosna modrzew platan buk cis jawor jarzębina akacja
źródło_światła: lampa latarka lampka żyrandol żarówka reflektor latarnia lampka
organ: wątroba płuco serce trzustka żołądek nerka macica jajowód nasieniowód prostata śledziona
oddziały: kompania pluton batalion brygada armia dywizja pułk
napój_alkoholowy: piwo wino wódka dżin nalewka bimber wiśniówka cydr koniak wiśniówka
kot_drapieżny: puma pantera lampart tygrys lew ryś żbik gepard jaguar
metal: żelazo złoto srebro miedź nikiel cyna cynk potas platyna chrom glin aluminium
samolot: samolot odrzutowiec awionetka bombowiec myśliwiec samolocik helikopter śmigłowiec
owoc: jabłko gruszka śliwka brzoskwinia cytryna pomarańcza grejpfrut porzeczka nektaryna
pościel: poduszka prześcieradło kołdra kołderka poduszeczka pierzyna koc kocyk pled
agd: lodówka kuchenka pralka zmywarka mikser sokowirówka piec piecyk piekarnik'''

In [19]:
with open('word_embedings_file.txt_a_nospoil.txt', "w") as f1, open('word_embedings_file_b_nospoil.txt', 'w') as f2, open('word_embedings_file_a_spoil.txt', 'w') as f3, open('word_embedings_file_b_spoil.txt', 'w') as f4:
    for line in clusters_txt.split('\n'):
        words = line.split(':')[1].split()
        for word in words:
            embedding_a_nospoil = get_context_free_token_embedding(word)
            embedding_b_nospoil = get_context_token_embedding(word)
            embedding_a_spoil = get_context_free_token_embedding(spoil(word))
            embedding_b_spoil = get_context_token_embedding(spoil(word))
            f1.write(f"{word} " + " ".join(map(str, embedding_a_nospoil)) + "\n")
            f2.write(f"{word} " + " ".join(map(str, embedding_b_nospoil)) + "\n")
            f3.write(f"{word} " + " ".join(map(str, embedding_a_spoil)) + "\n")
            f4.write(f"{word} " + " ".join(map(str, embedding_b_spoil)) + "\n")

In [ ]:
# !rm word_embedings_file.txt

In [45]:
# !export TOKENIZERS_PARALLELISM=true
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
!echo $TOKENIZERS_PARALLELISM

true


### a) nospoil

In [46]:
!cp word_embedings_file_a_nospoil.txt word_embedings_file.txt
!python3 word_emb_evaluation.py
!rm word_embedings_file.txt

PROBLEMS: 0.0
Start
TOTAL SCORE: 0.716038


### b) nospoil

In [47]:
!cp word_embedings_file_b_nospoil.txt word_embedings_file.txt
!python3 word_emb_evaluation.py
!rm word_embedings_file.txt

PROBLEMS: 0.0
Start
TOTAL SCORE: 0.593276


### a) spoil

In [48]:
!cp word_embedings_file_a_spoil.txt word_embedings_file.txt
!python3 word_emb_evaluation.py
!rm word_embedings_file.txt

PROBLEMS: 0.0
Start
TOTAL SCORE: 0.603584


### b) spoil

In [49]:
!cp word_embedings_file_b_spoil.txt word_embedings_file.txt
!python3 word_emb_evaluation.py
!rm word_embedings_file.txt

PROBLEMS: 0.0
Start
TOTAL SCORE: 0.548334
